<a href="https://colab.research.google.com/github/nasim-fani/ensemble_lfw/blob/main/Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip  "/content/gdrive/MyDrive/project.zip"

In [ ]:
import re
import os
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [ ]:
def readtxtfile(path):
  file = open(path, 'r')
  Line = file.read()
  Line = Line.replace("[","")
  Line = Line.replace("]","")
  data = np.fromstring(Line, sep=' ')
  return data

In [ ]:
def createData(path):
  file1 = open(path, 'r')
  Lines = file1.readlines()[1:]
  count = 0
  data = np.empty([len(Lines),512])
  labels = np.zeros(len(Lines))
  for line in Lines:
    line = line.strip()
    line = re.split(r'\t+', line)
    # print("Line{}: {}".format(count, line))
    for dirpath, dirnames, filenames in os.walk("/content/lfw/"+line[0]):
        for filename in filenames:
          # first file
          if filename == line[0]+"_"+f"{int(line[1]):04}"+".txt":
              path1 = os.path.join(dirpath, filename)
              val1 = readtxtfile(path1)
              
          else:#second file
            if len(line)==3:#3tayi
              if filename == line[0]+"_"+f"{int(line[2]):04}"+".txt":
                path2 = os.path.join(dirpath, filename)
                val2 = readtxtfile(path2)
                
    if len(line)==4:#4tayi
          for dirpath1, dirnames1, filenames1 in os.walk("/content/lfw/"+line[2]):
              for filename1 in filenames1:
                  if filename1 == line[2]+"_"+f"{int(line[3]):04}"+".txt":
                    path2 = os.path.join(dirpath1, filename1)
                    val2 = readtxtfile(path2)
                    
    if len(line)==3:
      label = 1
    else: label = 0
    # row = np.concatenate((val1, val2), axis=None)
    # row = np.sum((val1, val2))
    row = np.subtract(val1,val2)
    data[count] = row
    labels[count] = label
    count += 1
  return data,labels

In [ ]:
train_data, train_labels = createData('pairsDevTrain.txt')
test_data, test_labels = createData('pairsDevTest.txt')

**Random forest**

In [ ]:
clf = RandomForestClassifier(n_estimators=300,max_depth=50,max_features=0.4, warm_start=True)
clf.fit(train_data, train_labels)
clf.score(test_data, test_labels)

0.824



*   sum, no param -> 0.491
*   concat, no param -> 0.632
*   concat, n_estimators=300,max_depth=50,max_features=0.4 -> 0.688
*   subtract, n_estimators=300,max_depth=50,max_features=0.4 -> 0.817







 **AdaBoost**

In [ ]:
#Adaboost
dt_stump = DecisionTreeClassifier(max_depth=5, min_samples_leaf=1)
dt_stump.fit(train_data, train_labels)


clf = AdaBoostClassifier(n_estimators=300,base_estimator=dt_stump)
clf.fit(train_data, train_labels)
clf.score(test_data, test_labels)

0.734





*   sum, no param -> 0.502
*   concat, no param -> 0.522
*   subtract, n_estimators=300 ->0.788



